<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con NetworkX (II)</div>
<!-- Paulo Villegas, 2018 - v. 2.0 -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

%matplotlib inline
plt.style.use('bmh')
mpl.rcParams['figure.figsize'] = (16, 10)

# Datos

Lee datos de ficheros CSV a DataFrames de Pandas

In [ ]:
airports = pd.read_csv('DATA/openflights/airports.csv.bz2', escapechar="\\", na_values='\\N')
routes = pd.read_csv('DATA/openflights/routes.csv.bz2', na_values='\\N')

In [ ]:
len(airports), len(routes)

In [ ]:
airports.head()

# Red

Crea la red:

* usamos el DataFrame de rutas (los enlaces) para crear directamente el grafo. Basta indicar las columnas que corresponden al nodo de origen y de destino, y columnas adicionales que queramos añadir como atributos del enlace
* añadimos los datos sobre aeropuertos como atributos de los nodos del grafo

In [ ]:
# Crea la red desde el DataFrame de enlace. Aparte de origen & destino, añadimos 3 columnas como atributos
vuelos = nx.from_pandas_edgelist(routes,'SourceAirport','DestAirport', ['Airline', 'Codeshare', 'Stops'])

# Añadimos atributos de los nodos iterando sobre el DataFrame de los aeropuertos 
# (el id del nodo es el código IATA) del aeropuerto
for ap in airports.itertuples(index=False):
    vuelos.add_node(ap.IATA, **ap._asdict())

# Métricas

In [ ]:
nx.average_clustering(vuelos)

## Grados

Una función de NetworkX calcula el grado de todos los nodos

In [ ]:
dg = nx.degree(vuelos)

In [ ]:
dg['ATL']

Vamos a representar el grado de cada nodo, ordenado de mayor a menor grado. La estructura original de `dg` es una lista de tuplas; lo movemos a un diccionario ordenado (por grado)

In [ ]:
from collections import OrderedDict
from operator import itemgetter

# Crea el diccionario de grados, ordenando por el grado del aeropuerto, en orden descendente
dg_ord = OrderedDict( ((k,v) for k, v in sorted(iter(dg), key=itemgetter(1), reverse=True)))

In [ ]:
dg_ord

In [ ]:
# Pintamos un diagrama de barras
fig, ax = plt.subplots(figsize=(16,10))
ax.bar(range(len(dg_ord)), dg_ord.values(), align="center")

# En el eje X rotulamos con el código del aeropuerto, pero pintamos solo uno de cada 70 para que no se amontonen
ax.set_xlim(0, len(dg_ord));
ax.xaxis.set_ticks(np.arange(0, len(dg_ord), 70))
ax.set_xticklabels(list(dg_ord.keys())[::70], fontdict={'rotation': 'vertical'});

In [ ]:
# Hacemos zoom en los primeros 50 aeropuertos
from IPython.display import display

ax.set_xlim(0, 50)
ax.xaxis.set_ticks(np.arange(0, 50, 1))
ax.set_xticklabels(list(dg_ord.keys())[:50], fontdict={'rotation': 'vertical'});
display(fig)

In [ ]:
airports.loc[airports.IATA == 'IST']

## Distribución de grados

Calculamos la distribución de frecuencias para los grados de los nodos. 

O, dicho de otra manera, cómo de frecuentes son los aeropuertos con muchas rutas frente a los aeropuertos con pocas rutas.

In [ ]:
import seaborn as sns

In [ ]:
# Get all degree values
dg_values = [g[1] for g in dg]

np.seterr(all='warn');

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

sns.histplot(dg_values, ax=ax)
#sns.rugplot(dg_values, ax=ax, color='red', expand_margins=True, height=0.005)

ax.set_xlabel('número de conexiones');
ax.set_ylabel('número de aeropuertos');

In [ ]:
fig, ax = plt.subplots( figsize=(16,8))

sns.histplot(dg_values, ax=ax)

ax.set_xscale('symlog')
ax.set_yscale('symlog')
ax.set_xlim([1, 250]);

In [ ]:
fig, ax = plt.subplots( figsize=(16, 8))

ax = sns.histplot(dg_values, kde=True, line_kws={'linewidth': 4}, ax=ax)
regline = ax.get_lines()[0]
regline.set_color('red')

ax.set_xscale('symlog')
ax.set_yscale('symlog')
ax.set_xlim([1, 250]);